## Abstract Attributes

Not working yet.

In [308]:
def abstractattrs(*attrs):
    def wrapper(cls):
        
        def method(cls, *args, **kwargs):
            super().__init_subclass__(*args, **kwargs)
            for attr in attrs:
                if not hasattr(cls, attr):
                    raise RuntimeError(f'Class must have attribute {attr}.') 
                    
        setattr(cls, '__init_subclass__', classmethod(method))
        return cls
    return wrapper              

In [309]:
@abstractattrs('name', 'age')
class Base:
    def __init__(self, age=7):
        self.age = age 

In [310]:
class Child(Base):
    a = 'a'

In [311]:
def foo(a, b, **kwargs):
    return c

In [312]:
foo(3, 4, c=5)

## LSH

In [30]:
from datasketch import lsh, MinHashLSH, MinHash, MinHashLSHForest

from htools import *

In [2]:
set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)

In [3]:
def ngrams(word, n=3, step=1, drop_last=False):
    """To get non-overlapping sequences, pass in same value for `step` as `n`."""
    stop = max(1, step+len(word)-n)
    ngrams_ = []
    for i in range(0, stop, step):
        ngrams_.append(word[i:i+n])
    if drop_last and len(ngrams_[-1]) < n: ngrams_ = ngrams_[:-1]
    return ngrams_

In [35]:
w = 'dinosaur'
tw = 'I'
ws = ['dino', 'rhino', 'dog', 'cat', 'lion', 'tyrannosaurus rex',
      'tiger', 'owl', 'rhinocerous', 'kino']
s = 'The mailman was running late today, possibly due to the heavy '\
    'snowfall from the previous night.'

In [6]:
ngrams(w)

['din', 'ino', 'nos', 'osa', 'sau', 'aur']

In [7]:
ngrams(w, 2)

['di', 'in', 'no', 'os', 'sa', 'au', 'ur']

In [9]:
ngrams(w, 3, 3)

['din', 'osa', 'ur']

In [12]:
ngrams(w, 3, 3, True)

['din', 'osa']

In [37]:
ngrams(s)

['The',
 'he ',
 'e m',
 ' ma',
 'mai',
 'ail',
 'ilm',
 'lma',
 'man',
 'an ',
 'n w',
 ' wa',
 'was',
 'as ',
 's r',
 ' ru',
 'run',
 'unn',
 'nni',
 'nin',
 'ing',
 'ng ',
 'g l',
 ' la',
 'lat',
 'ate',
 'te ',
 'e t',
 ' to',
 'tod',
 'oda',
 'day',
 'ay,',
 'y, ',
 ', p',
 ' po',
 'pos',
 'oss',
 'ssi',
 'sib',
 'ibl',
 'bly',
 'ly ',
 'y d',
 ' du',
 'due',
 'ue ',
 'e t',
 ' to',
 'to ',
 'o t',
 ' th',
 'the',
 'he ',
 'e h',
 ' he',
 'hea',
 'eav',
 'avy',
 'vy ',
 'y s',
 ' sn',
 'sno',
 'now',
 'owf',
 'wfa',
 'fal',
 'all',
 'll ',
 'l f',
 ' fr',
 'fro',
 'rom',
 'om ',
 'm t',
 ' th',
 'the',
 'he ',
 'e p',
 ' pr',
 'pre',
 'rev',
 'evi',
 'vio',
 'iou',
 'ous',
 'us ',
 's n',
 ' ni',
 'nig',
 'igh',
 'ght',
 'ht.']

In [13]:
def lsh_hash_word(word, num_perm=128, **ngram_kwargs):
    mhash = MinHash(num_perm=num_perm)
    for ng in ngrams(word, **ngram_kwargs):
        mhash.update(ng.encode('utf8'))
    return mhash

In [16]:
m = lsh_hash_word(w)

In [22]:
forest = MinHashLSHForest(num_perm=128)

for i, word in enumerate(ws):
    forest.add(i, lsh_hash_word(word))
forest.index()

In [24]:
forest.query(lsh_hash_word(w), 3)

[8, 0, 5]

In [26]:
ws

['dino',
 'rhino',
 'dog',
 'cat',
 'lion',
 'tyrannosaurus rex',
 'tiger',
 'owl',
 'rhinocerous',
 'kino']

In [28]:
idx = forest.query(lsh_hash_word(w), 3)
idx

[8, 0, 5]

In [32]:
select(ws, idx)

['dino', 'tyrannosaurus rex', 'rhinocerous']